In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
""" Load the data """

df = pd.read_csv('data/processed/traffic_volume_2021_plus.csv')  # Update with your data source

# Rename columns for clarity
df.rename(columns={'yr': 'year', 'm': 'month', 'd': 'day', 'hh': 'hour', 'mm': 'minute'}, inplace=True)

# Create timestamp
df['timestamp'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

# Handle outliers: Remove top 1% of vol to reduce skew
df = df[df['vol'] <= df['vol'].quantile(0.99)]

In [ ]:
""" Feature Engineering """

# Derived time features
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
df['is_rush_hour'] = df['hour'].isin([7, 8, 9, 16, 17, 18]).astype(int)
df['month'] = df['timestamp'].dt.month
df['season'] = df['timestamp'].dt.month % 12 // 3  # 0: Winter, 1: Spring, etc.

# Lagged feature (previous hour's volume by segment, borough, direction)
df['vol_lag1'] = df.groupby(['segmentid', 'boro', 'direction'])['vol'].shift(1)

# Log-transform target variable
df['log_vol'] = np.log1p(df['vol'])  # log(1 + x) to handle zeros

# Drop rows with NaN values from lagged features
df = df.dropna()

# Define features
numerical = ['hour', 'minute', 'dayofweek', 'is_weekend', 'is_rush_hour', 'month', 'season', 'vol_lag1']
categorical = ['boro', 'direction', 'segmentid']

In [ ]:
""" Train-Test Split """

# Define X and y
X = df[numerical + categorical]
y = df['log_vol']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
""" Preprocessing pipeline with RobustScaler for numerical features """

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical),
        ('num', RobustScaler(), numerical)
    ]
)

In [ ]:
""" Random Forest Modeling with tuning """

# Random Forest pipeline
rf_model = make_pipeline(
    preprocessor,
    RandomForestRegressor(random_state=42, n_jobs=-1)
)

# Hyperparameter tuning
param_grid = {
    'randomforestregressor__n_estimators': [100, 200],
    'randomforestregressor__max_depth': [10, 15, None],
    'randomforestregressor__min_samples_split': [2, 5],
    'randomforestregressor__min_samples_leaf': [1, 2]
}
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model and predictions
print(f"Best parameters: {grid_search.best_params_}")
y_pred_log = grid_search.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)
rf_mae = mean_absolute_error(y_true, y_pred)
print(f"Random Forest MAE: {rf_mae:.2f}")